In [105]:
import sqlite3
import numpy as np
import pandas as pd
from sqlite3 import Error
import math
import blpapi
import pdblp
from datetime import date
from Database import PySQLite

DB = PySQLite("test1228v1.db")
currentYear = 2020.75
currentQuarter = 4

pd.options.display.float_format = '{:,.2f}'.format
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})






class CorporateCredit():    
    def __init__(self,name):     
        self.name = name
    
    def test(self):
        print('test')
        
    def Sensitize(self,table,field,amount,target):        
        #returns [target] after changing [field] by [amount]
        # do for first period, not sure if useful for future periods, instead should be able to call FCF_Proj
        # on sensitized variables
        # generic case (not sector specific) - can change revenue, vcost, fcost, ebitda, capex, int cost
        # targets are EBITDA and FCF (add leverage to proj function)
        
        # WC = DB.GetRecord(table,'Name','VCost',self.name) - should have error check for when field doesn't exist
        # also should add WC and tax to midstream template even if zerod out
        
        Rev = DB.GetRecord(table,'Name','Revenue',self.name)
        FCost = DB.GetRecord(table,'Name','FCost',self.name)
        VCost = DB.GetRecord(table,'Name','VCost',self.name)
        Capex = DB.GetRecord(table,'Name','Capex',self.name)
        Intexp = DB.GetRecord(table,'Name','AvgIntCost',self.name)
        Debt = DB.GetRecord(table,'Name','TotalDebt',self.name)
        
        Interest = Intexp*Debt


        EBITDA = Rev*(1 + (amount*(field=='Revenue'))) - FCost*(1 + (amount*(field=='FCost'))) - VCost*(1 + (amount*(field=='VCost')))    
        FCF = EBITDA - Capex*(1 + (amount*(field=='Capex'))) - Interest*(1 + (amount*(field=='Debt')))
        
#         print('Rev',Rev,'FCost',FCost,'VCost',VCost,'EBITDA',EBITDA,'Interest',Interest,'Capex',Capex,'FCF',FCF)
        
        TargetDict={'EBITDA':EBITDA,'FCF':FCF}
        
        
        
        return(TargetDict[target])
        
        
        
        
        
        
    def makeDateColumn(self,qtrs):
        yearColumns = ['Q'+str(currentQuarter)+'/'+str(math.floor(currentYear))]
        for x in range (qtrs):
            yearColumns.append('Q'+str(((currentQuarter+x) % 4)+1)+'/'+str(math.floor(currentYear+(x+1)*0.25)))
        
        return yearColumns
    
    def FCF_Proj(self,qtrs):
        # add gross and net leverage 
        FCF = ['Revenue','Variable Costs','Fixed Costs','EBITDA', 'Less: Capex','Less: Interest', 'Less: Working Cap', 'Less: Taxes','Less: Pfd Div','Less: Common Div', 'FCF', 'Beginning Cash', 'Ending Cash'] 

        revArray = np.array([self.CurrentFCF['rev']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
        vcostsArray = np.array([self.CurrentFCF['var_costs']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
        fcostsArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['f_costs'],dtype=int)
        ebitdaArray = revArray-vcostsArray-fcostsArray
        capexArray = revArray * self.CurrentFCF['capex/rev']
        
        interestArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['interest'],dtype=int)               
        pfdArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['pfd_distro'],dtype=int)                
        CommonArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['common_distro'],dtype=int)                
        
        wcArray = revArray * self.CurrentFCF['WC/rev']
        taxArray = (ebitdaArray - interestArray)*self.CurrentFCF['tax']  #to fix --> need D&A
        fcfArray = ebitdaArray - capexArray - wcArray - taxArray - interestArray - pfdArray - CommonArray 

        begCashArray = np.zeros(qtrs+1,dtype=int)
        begCashArray[0] = self.CurrentFCF['cash']
        endCashArray = np.zeros(qtrs+1,dtype=int)
        endCashArray[0] = fcfArray[0]+begCashArray[0]


        for i in range(1,qtrs+1):
            begCashArray[i] = endCashArray[i-1]
            endCashArray[i] = begCashArray[i] + fcfArray[i]


        FCF_statement=[revArray,vcostsArray,fcostsArray,ebitdaArray,capexArray,interestArray,wcArray,taxArray,pfdArray,CommonArray,fcfArray,begCashArray,endCashArray]
        FCF_df=pd.DataFrame(FCF_statement,columns = self.makeDateColumn(qtrs), index = FCF, dtype=int)


        print(fcfArray)
        return FCF_df
    
#     def SensitizeSector(self,table):
#         df=DB.DBdescription(table)
        
#         for rows in df.iterrows():
            
            
#         x=df.apply(self.Sensitize,args=['test_table1','FCost',-0.10,'EBITDA'])
        
        

def main():
    ETP=CorporateCredit('ETP')
    


    df=DB.DBdescription('test_table1')  
    df=df.iloc[:-1]
    n=0
    list=[]
    namelist=[]
    for rows in df.iterrows():
        x=CorporateCredit(df.iloc[n]['Name'])
        namelist.append(df.iloc[n]['Name'])
        list.append(x.Sensitize('test_table1','FCost',-0.20,'EBITDA'))        
        n+=1
    
    list2=[namelist,list]
    df2=pd.DataFrame(list2)
    print(df2)
        
        
        
#         x=CorporateCredit(df['Name'])
        
#         x.Sensitize('test_table1','FCost',-0.10,'EBITDA')
        
        
        
        
        
#     x=df.apply(lambda x: Sensitize('test_table1','FCost',-0.10,'EBITDA'),axis=1)
    
               
#     Sensitize(self,table,field,amount,target)
#     ETP.CurrentFCF = {'rev': 100, 'rev_g': 0.05, 'var_costs': 2.7, 'f_costs': 3.9, 'cash': 4, 'capex/rev': 5,'interest':6,'pfd_distro':7,'common_distro':8,'WC/rev': 9, 'tax':1000}
# #     ETP.FCF_Proj(5)
#     print(ETP.Sensitize('test_table1','FCost',-0.10,'EBITDA'))
#     print(ETP.Sensitize('test_table1','FCost',-0.10,'FCF'))
#     print(ETP.Sensitize('test_table1','Capex',0.10,'FCF'))
#     print(ETP.Sensitize('test_table1','Debt',0.20,'FCF'))
    
#     print(x)
    
    
main()

     

         0      1      2      3      4      5
0      ETP    PAA    DCP    WES   ENLK  ONEOK
1 2,819.80 474.00 198.40 466.52 283.00 744.80
